In [23]:
%run input/Format.ipynb
import ROOT as root
from array import array
from math import sin
from ROOT import gRandom
from ROOT import TFile
from ROOT import TPad, TLegend
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

In [24]:
iOption0 = 0

In [25]:
isigma = 2
N_files = 1
N_centr = 6
N_hists = 4
N_layers = 6
N_ilyaers = 1
is_sigmalised = 0
is_e = 1
iOption1 = 0
centralities = ["1 layer pos","1 layer neg","2 layer pos","2 layer neg","3 layer pos","3 layer neg","20-40%","40-60%","60-80%","80-93%"]
centrality = [1,1,2,2,3,3,4,4,5,5]
layer_bins = [35,36,40,41,47,48,52,53,59,60,64,65]
layer_bins = [46,46,48,48,50,50,51,51,53,53,55,55]
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4]
file_path="input/rg/"
file_names="my-24th_QA_19192_80runs_rg0"
hist_select_3D_names = ["dphi_hist","dthe_hist","sdphi_hist","sdthe_hist"]
if is_e == 1: hist_select_3D_names = ["dphi_hist_el","dthe_hist_el","sdphi_hist_el","sdthe_hist_el"]
iOptames=["phi","the"]

In [26]:
hists_emc_read = []

for iFile in range(N_files):
    print(file_path+file_names+f"{iFile}.root")
    infile = root.TFile.Open(file_path+file_names+f"{iFile}.root", "read")
    hists_emc_read0 = []
    for icentr in range(N_centr):
        hist_select_3D0 = []
        for ihist in range(len(hist_select_3D_names)):
            hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{icentr}"))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
        hists_emc_read0.append(hist_select_3D0)
    hists_emc_read.append(hists_emc_read0)

infile.Close()

input/rg/my-24th_QA_19192_80runs_rg00.root


In [27]:
pt = array( 'd' )
pt_boarders = array( 'd', [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.4,2.0,2.8,4.9])
pt_err = array( 'd' )
N_pt = len(pt_boarders) - 1
for i in range(N_pt):
    pt.append( (pt_boarders[i]+pt_boarders[i+1])/2)
    pt_err.append((pt_boarders[i+1]-pt_boarders[i])/2)
print("pt = ", pt,"\npt_boarders = ",pt_boarders)
pt[0]=0.225

pt =  array('d', [0.25, 0.35, 0.45, 0.55, 0.6499999999999999, 0.75, 0.8500000000000001, 0.95, 1.2, 1.7, 2.4, 3.85]) 
pt_boarders =  array('d', [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.4, 2.0, 2.8, 4.9])


In [28]:
boards_tot = [[-0.099,0.099],[-0.099,0.099]]
boards_fit = [[-0.011,0.012],[-0.011,0.012]]
boards_fit2 = [[-0.03,-0.01],[0.015,0.035],[-0.09,-0.03],[0.035,0.09],[-0.06,-0.025],[0.03,0.065]]
boards_fit3 = [[-0.02,0.02],[-0.02,0.02]]
if iOption1==1:
    boards_tot = [[-5,5],[-5,5]]
    boards_fit = [[-1,1],[-1,1]]
if is_sigmalised == 1:
    boards_tot = [[-2,2],[-2,2]]
    boards_fit = [[-1,1],[-1,1]]
    boards_fit2 = [[0.6,1.5],[0.6,1.5]]
    boards_fit3 = [[-2,2],[-2,2]]

In [29]:
def emc_fitter(InHist, ilayer, seed=1, icentr=0, sigma=2, fits_mean=0, fits_sigma=0):
    N_gaus=2
    seed0 = seed
    seed=seed*N_pt*icentr*100

    hist_pt_bin = InHist.ProjectionZ("kek")

    central_bean_prev = (0.002*layer_bins[2*ilayer]-0.001-0.1)
    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )
 
    for p in range(N_pt):
        c1 = root.TCanvas(f"c1{seed+p}",f"c1{seed+p}",720,360)

        m2_name=str(icentr)+"/"+str(ilayer)+"dynamic_"+str(round(pt[p],1))

        emc_var_porj = InHist.ProjectionX(f"emc{seed+p}_{seed+p}_{icentr}_{ilayer}",layer_bins[2*ilayer],layer_bins[2*ilayer+1],hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1)
        emc_var_porj.SetLineColor(1)

        par_tot = array( 'd', (3*N_gaus+3)*[2.] )
        g_gaus= root.TF1( f'g_gaus_{seed+p}_{icentr}_{ilayer}', 'gaus',  boards_fit[int(icentr%2)][0]+central_bean_prev, boards_fit[int(icentr%2)][1]+central_bean_prev)
        g_gaus2= root.TF1( f'g_gaus_2_{seed+p}_{icentr}_{ilayer}', 'gaus',  boards_fit2[icentr][0]+central_bean_prev, boards_fit2[icentr][1]+central_bean_prev)#[0]/[2]/sqrt(2*3.14159)*exp(-0.5*pow((x-[1])/[2],2))
        g_pol = root.TF1( f'g_pol{seed+p}_{icentr}_{ilayer}', '[0]+[1]*x*(abs(x)/x-1)+[2]*x*(abs(x)/x+1)',  boards_tot[int(icentr%2)][0],boards_tot[int(icentr%2)][1])
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_pos{seed+p}_{icentr}_{ilayer}', 'gaus(0)+gaus(3)+[6]+[7]*x*(abs(x)/x-1)+[8]*x*(abs(x)/x+1)',boards_tot[int(icentr%2)][0], boards_tot[int(icentr%2)][1])
        Format_Func(total,2,6)

        g_gaus.SetParameter(1,0.)
        g_gaus.SetParameter(2,boards_fit[1][1])
        g_gaus2.SetParameter(1,0.5*(boards_fit2[icentr][0]+boards_fit2[icentr][1])+central_bean_prev)
        g_gaus2.SetParameter(2,0.5*(boards_fit2[icentr][1]-boards_fit2[icentr][0]))
        g_gaus2.SetParLimits(0,1,10000000)
        g_gaus2.SetParLimits(1,boards_fit2[icentr][0]+central_bean_prev, boards_fit2[icentr][1]+central_bean_prev)

        emc_var_porj.Draw()
        emc_var_porj.Fit( g_gaus, 'QR')

        if iOption0 == 4:
            Mean_local.append(g_gaus.GetParameter(1))
            Sigma_local.append(g_gaus.GetParameter(2))
            Mean_local_err.append(g_gaus.GetParError(1)+0.05*g_gaus.GetParameter(2))
            Sigma_local_err.append(g_gaus.GetParError(2)+0.05*g_gaus.GetParameter(2))
            c1.SaveAs( "output/d_delta_fits/" + m2_name + ".png" )
            continue

        emc_var_porj.Fit( g_gaus2, 'QR+' )
        emc_var_porj.Fit( g_pol, 'QR+' )

        par_gaus = g_gaus.GetParameters()
        par_gaus2 = g_gaus2.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5] = par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2]
        par_tot[6], par_tot[7], par_tot[8] = par_pol[0], par_pol[1], par_pol[1]

        total.SetParameters( par_tot )
        total.SetParLimits(1,boards_tot[int(icentr%2)][0]/10+central_bean_prev,boards_tot[int(icentr%2)][1]/10+central_bean_prev)
        total.SetParLimits(2,boards_fit[int(icentr%2)][1]/10,2)
        total.SetParLimits(3,par_gaus2[0]*0.49, par_gaus2[0]*2)
        total.SetParLimits(4,par_gaus2[1]-0.01, par_gaus2[1]+0.01)
        total.SetParLimits(5,par_gaus2[2]*0.49, par_gaus2[2]*2)
        #total.SetParLimits(3,1,1000000)
        if ilayer>-1:
            total.SetParLimits(1,boards_tot[int(icentr%2)][0],boards_tot[int(icentr%2)][1])
        #if iOption0%2==1: total.FixParameter(3,0.)
        
        emc_var_porj.Draw()
        emc_var_porj.Fit( total, 'QR' )

        par_tot = total.GetParameters()

        g_gaus= root.TF1( f'g_gaus_4_{seed+p}_{icentr}_{ilayer}', 'gaus',  par_tot[1]-2*par_tot[2], par_tot[1]+2*par_tot[2])
        g_gaus2= root.TF1( f'g_gaus_5_{seed+p}_{icentr}_{ilayer}', 'gaus',  par_tot[4]-2*par_tot[5], par_tot[4]+2*par_tot[5])

        Format_Func(g_gaus,3,4)
        Format_Func(g_gaus2,3,root.kGreen+2)

        par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2] = par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5]
        par_pol[0], par_pol[1], par_pol[2] = par_tot[6], par_tot[7], par_tot[8]
        g_gaus2.SetParameters(par_gaus2)
        g_gaus.SetParameters(par_gaus)
        g_pol.SetParameters(par_pol)

        g_gaus.Draw('same')
        g_gaus2.Draw('same')
        g_pol.Draw('same')

        if iOption0<4:
            Mean_local.append(total.GetParameter(1))
            Sigma_local.append(total.GetParameter(2))
            Mean_local_err.append(total.GetParError(1)+0.05*total.GetParameter(2))
            Sigma_local_err.append(total.GetParError(2)+0.05*total.GetParameter(2))


        int_fg_local.append(g_gaus.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
        int_bg_local.append(g_pol.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
            
        #m2_pos.Draw("same") m2_neg.Draw("same")

        c1.SaveAs( "output/dynamic_delta_fits/" + m2_name + ".png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, int_fg_local, int_bg_local]


In [30]:
def emc_centr_fitter(InHist, layers, seed=1, sigma=2, fits_mean=0, fits_sigma=0):
    out_hists = []
    for icentr in range(N_centr):
        out_hists1 = []
        for ilayer in layers:
            out_hists1.append(emc_fitter(InHist[seed][icentr][iOption0+iOption1*2], ilayer, seed, icentr, sigma, fits_mean, fits_sigma))
        out_hists.append(out_hists1)
    return out_hists

In [31]:
import multiprocess as mp

Ntr = N_files

pool = mp.Pool(Ntr)
output_array = pool.starmap(emc_centr_fitter, [(hists_emc_read, [0,1,2,3,4,5], i, isigma) for i in range(Ntr)])
pool.close()

In [32]:
Means, Mean_errs, Sigmas, Sigma_errs, Int_FG, Int_BG = [], [], [], [], [], []
for ifile in range(Ntr):
    Means1, Mean_errs1, Sigmas1, Sigma_errs1, Int_FG1, Int_BG1 = [], [], [], [], [], []
    for icentr in range(N_centr):
        Means0, Mean_errs0, Sigmas0, Sigma_errs0, Int_FG0, Int_BG0 = [], [], [], [], [], []
        for ilayer in range(N_layers):
            Means0.append(output_array[ifile][icentr][ilayer][0])
            Mean_errs0.append(output_array[ifile][icentr][ilayer][1])
            Sigmas0.append(output_array[ifile][icentr][ilayer][2])
            Sigma_errs0.append(output_array[ifile][icentr][ilayer][3])
            Int_FG0.append(output_array[ifile][icentr][ilayer][4])
            Int_BG0.append(output_array[ifile][icentr][ilayer][5])
        Means1.append(Means0)
        Mean_errs1.append(Mean_errs0)
        Sigmas1.append(Sigmas0)
        Sigma_errs1.append(Sigma_errs0)
        Int_FG1.append(Int_FG0)
        Int_BG1.append(Int_BG0)
    Means.append(Means1)
    Mean_errs.append(Mean_errs1)
    Sigmas.append(Sigmas1)
    Sigma_errs.append(Sigma_errs1)
    Int_FG.append(Int_FG1)
    Int_BG.append(Int_BG1)

In [33]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
fit_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for icentr in range(N_centr):
        fit_functions1 = []
        for ilayer in range(N_layers):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_{ifile}_{ilayer}_{icentr}","[0]+[1]*exp([2]*x)",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.0015)
            fit_functions1[-1].SetParameter(1,0.02)
            fit_functions1[-1].SetParameter(2,-2)
            fit_functions1[-1].SetParLimits(2,-8.126,-0.023)
            fit_functions1[-1].SetParLimits(0,0.001,0.002)
            if icentr<2 : fit_functions1[-1].SetParLimits(2,-5.5+0.5,-4.4-0.5)
            elif icentr<4 : fit_functions1[-1].SetParLimits(2,-3.0+0.45,-2.0-0.45)
            else : 
                fit_functions1[-1].SetParLimits(2,-3.0+0.45,-2.0-0.45)
                fit_functions1[-1].SetParLimits(0,0.001,0.002)
            if iOption0%2==1: 
                fit_functions1[-1].SetParLimits(2,-3.126,-1.23)
                fit_functions1[-1].SetParameter(1,0.02)
                fit_functions1[-1].SetParLimits(1,0.001,0.6)
                fit_functions1[-1].SetParameter(0,0.005)
                fit_functions1[-1].SetParLimits(0,0.001,0.006)
            fit_functions1[-1].SetLineColor(colors[ilayer])
        fit_functions0.append(fit_functions1)
    fit_functions.append(fit_functions0)

In [34]:

print(Sigmas[0][0][0],Sigmas[0][3][0])
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1500,900)
    c3.Divide(3,2)
    g_sigma = [[[[]]*N_layers]*N_centr]*N_files
    min, max = 0., 0.01
    if iOption1==1: max, min = 4.99, 0.01
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for icentr in range(N_centr):
        c3.cd(icentr+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,centralities[icentr]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):
            g_sigma[ifile][icentr][ilayer].append(root.TGraphErrors(N_pt, pt, Sigmas[ifile][icentr][ilayer],pt_err,Sigma_errs[ifile][icentr][ilayer]))

            Format_Graph(g_sigma[ifile][icentr][ilayer][-1],20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
            g_sigma[ifile][icentr][ilayer][-1].Fit(fit_functions[ifile][icentr][ilayer],"Q","")
            g_sigma[ifile][icentr][ilayer][-1].Draw('P')
            central_bean_prev = round(0.002*layer_bins[2*ilayer]-0.001-0.1,3)
            legends[-1].AddEntry(g_sigma[ifile][icentr][ilayer][-1],f"phi_prev {central_bean_prev}","p")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/sigma_{ifile}{iOption0}{iOption1}.png")

array('d', [0.004557076526521889, 0.0031231400544969815, 0.002574166110568066, 0.0022795434037945297, 0.0021028772253901895, 0.0019903530876901054, 0.0019215982826221434, 0.0018718526057825442, 0.001788931237176471, 0.001698479275588667, 0.0018582404042979913, 0.003711835416813189]) array('d', [0.007086330669434942, 0.00564815316880329, 0.0044861984254327344, 0.0038158413415780787, 0.0033410721057538766, 0.0030469130394130137, 0.0028040009574796924, 0.002619446673918646, 0.002381598801850846, 0.0019514612622586911, 0.007329024723110013, 0.0017491881357831487])


In [35]:
print(f"sigma_pt_params[{N_files}][{N_centr}][{N_layers}][3]="+"{")
for fit_functions_ifile in fit_functions:
    print("{")
    for fit_functions_icentr in fit_functions_ifile:
        print("{")
        for fits in fit_functions_icentr:
            print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
        print("},") 
    print("},")      
print("};")        

sigma_pt_params[1][6][6][3]={
{
{
{0.001786, 0.008022, -4.999998},
{0.001773, 0.007722, -4.900002},
{0.001718, 0.007426, -4.9},
{0.001727, 0.007042, -4.9},
{0.001782, 0.006928, -5.0},
{0.001847, 0.00612, -5.0},
},
{
{0.002, 0.006267, -5.0},
{0.001918, 0.006923, -5.0},
{0.001858, 0.007533, -5.0},
{0.001826, 0.007838, -4.9},
{0.001865, 0.007673, -4.9},
{0.001909, 0.007654, -5.0},
},
{
{0.001899, 0.010984, -2.450001},
{0.001798, 0.010718, -2.45},
{0.001675, 0.010814, -2.45},
{0.001669, 0.010454, -2.45},
{0.001804, 0.009611, -2.524435},
{0.001912, 0.008747, -2.55},
},
{
{0.001899, 0.008335, -2.55},
{0.00182, 0.0089, -2.55},
{0.001692, 0.009624, -2.45},
{0.001671, 0.009947, -2.45},
{0.001678, 0.010556, -2.45},
{0.00171, 0.01091, -2.45},
},
{
{0.002, 0.004434, -2.55},
{0.001481, 0.004981, -2.45},
{0.001264, 0.004148, -2.45},
{0.001238, 0.004025, -2.55},
{0.001197, 0.003772, -2.55},
{0.001243, 0.003457, -2.55},
},
{
{0.001407, 0.003133, -2.55},
{0.00123, 0.003614, -2.55},
{0.001231, 0.003825,

In [36]:
fit_mean_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for icentr in range(N_centr):
        fit_functions1 = []
        for ilayer in range(N_layers):
            fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{ilayer}_"+centralities[icentr],"[0]",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.029)
            fit_functions1[-1].SetLineColor(colors[ilayer])
        fit_functions0.append(fit_functions1)
    fit_mean_functions.append(fit_functions0)

In [37]:
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3",f"c3",500*3,450*2)
    c3.Divide(3,2)
    g_mean = [[[[]]*N_layers]*N_centr]*N_files
    min, max = -0.1, 0.1
    if iOption1==1: max, min = -1, 1
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for icentr in range(N_centr):
        c3.cd(icentr+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,centralities[icentr]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):

            g_mean[ifile][icentr][ilayer].append(root.TGraphErrors(N_pt, pt, Means[ifile][icentr][ilayer],pt_err,Mean_errs[ifile][icentr][ilayer]))
            
            Format_Graph(g_mean[ifile][icentr][ilayer][-1],20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
            g_mean[ifile][icentr][ilayer][-1].Fit(fit_mean_functions[ifile][icentr][ilayer],"Q","")
            g_mean[ifile][icentr][ilayer][-1].Draw('P')
            central_bean_prev = round(0.002*layer_bins[2*ilayer]-0.001-0.1,3)
            legends[-1].AddEntry(g_mean[ifile][icentr][ilayer][-1],f"phi_prev {central_bean_prev}","p")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{iOption1}.png")

In [38]:
print(f"mean_pt_params[{N_files}][{N_centr}][{N_layers}]="+"{")
for fit_functions_file in fit_mean_functions:
    print("{")
    for fit_functions_syst in fit_functions_file:
        print("{"+f"{round(fit_functions_syst[0].GetParameter(0),6)}, {round(fit_functions_syst[1].GetParameter(0),6)},{round(fit_functions_syst[2].GetParameter(0),6)}, {round(fit_functions_syst[3].GetParameter(0),6)}, {round(fit_functions_syst[4].GetParameter(0),6)}, {round(fit_functions_syst[5].GetParameter(0),6)}"+"},")
    print("},")
print("};")

mean_pt_params[1][6][6]={
{
{-0.01016, -0.005913,-0.001761, 0.000286, 0.004368, 0.008459},
{-0.008474, -0.004401,-0.000319, 0.001709, 0.005811, 0.009995},
{-0.011349, -0.006712,-0.002387, -0.000191, 0.003925, 0.00793},
{-0.008141, -0.004256,-4.5e-05, 0.002012, 0.006343, 0.010718},
{-0.012342, -0.007189,-0.002374, 0.015827, 0.018424, 0.00857},
{-0.008863, -0.004573,-0.01435, -0.015561, 0.006617, 0.011242},
},
};


In [39]:
dphi = array( 'd' )
dphi_err = array( 'd' )
dynamic_dphi = []
dynamic_dphi_err = []
for ilayer in range(N_layers):
    dphi.append( round(0.002*layer_bins[2*ilayer]-0.001-0.1,5))
    dphi_err.append(0.001/2)

for ifile in range(N_files):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    for icentr in range(N_centr):
        dynamic_dphi_icentr = array( 'd' )
        dynamic_dphi_err_icentr = array( 'd' )
        for ilayer in range(N_layers):
            dynamic_dphi_icentr.append(fit_mean_functions[ifile][icentr][ilayer].GetParameter(0))
            dynamic_dphi_err_icentr.append(fit_mean_functions[ifile][icentr][ilayer].GetParError(0))
        dynamic_dphi_ifile.append(dynamic_dphi_icentr)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_icentr)
    dynamic_dphi.append(dynamic_dphi_ifile)
    dynamic_dphi_err.append(dynamic_dphi_err_ifile)

print("dphi = ", dphi)
print(dynamic_dphi)
if iOption0 != 0:
    dynamic_dphi[0][5][2]/=100
    dynamic_dphi[0][5][3]=0.002
    dynamic_dphi[0][4][3]=-0.0005
    dynamic_dphi[0][4][4]/=5

dphi =  array('d', [-0.009, -0.005, -0.001, 0.001, 0.005, 0.009])
[[array('d', [-0.010160416283190154, -0.005913327229959849, -0.0017607876173421112, 0.0002862930062024621, 0.004368204374169803, 0.008458683256434008]), array('d', [-0.008474499253845118, -0.004401438694611164, -0.0003190791341586806, 0.0017090156761349895, 0.005810817337189484, 0.009994918847388234]), array('d', [-0.011348868032203734, -0.006712201213909242, -0.0023867078375517624, -0.00019094077410871171, 0.003924998508621335, 0.007929901684385704]), array('d', [-0.00814136118515366, -0.004255558649213124, -4.548267524168148e-05, 0.0020119063627053804, 0.006342809686286893, 0.010718121710147327]), array('d', [-0.012341790319701309, -0.007188618943478376, -0.002373862041092304, 0.015827101087134934, 0.01842387365277398, 0.008569563179067353]), array('d', [-0.00886298233662641, -0.004573293807789178, -0.014350231654084045, -0.015561359519644973, 0.006617434367136545, 0.011241769374655104])]]


In [40]:
for ifile in range(N_files):
    h3=[]
    c4 = root.TCanvas(f"c4",f"c4",600,450)
    g_mean_phi = [[[[]]*N_layers]*N_centr]*N_files
    min, max = -0.05, 0.05
    if iOption1==1: max, min = -1, 1
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(1):
        h3.append(Format_Pad_old( -0.02, 0.05, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h3[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for icentr in range(0,N_centr):

            g_mean_phi[ifile][icentr][ilayer].append(root.TGraphErrors(N_layers, dphi, dynamic_dphi[ifile][icentr],dphi_err,dynamic_dphi_err[ifile][icentr]))
            
            Format_Graph(g_mean_phi[ifile][icentr][ilayer][-1],20+int(icentr/4),1, colors[icentr], 3, colors[icentr],1,1)
            g_mean_phi[ifile][icentr][ilayer][-1].Fit("pol1","","")
            g_mean_phi[ifile][icentr][ilayer][-1].Draw('sameP')
            legends[-1].AddEntry(g_mean_phi[ifile][icentr][ilayer][-1],centralities[icentr],"p")

        legends[-1].Draw()
    c4.Draw()
    c4.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{iOption1}.png")

 FCN=0.0567144 FROM MIGRAD    STATUS=CONVERGED      74 CALLS          75 TOTAL
                     EDM=6.81833e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0          -7.86874e-04   2.11253e-04   1.06033e-07  -9.02587e-02
   2  p1           1.03290e+00   3.53812e-02   1.77578e-05   8.93779e-04
 FCN=0.0230902 FROM MIGRAD    STATUS=CONVERGED      43 CALLS          44 TOTAL
                     EDM=1.17804e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           7.19895e-04   2.09662e-04   1.03509e-07  -2.15956e+00
   2  p1           1.02486e+00   3.51152e-02   1.73228e-05  -4.98478e-03
 FCN=0.677261 FROM MIGRAD    STATUS=CONVERGED      46 CALLS          47 TOTAL
                     EDM

In [41]:
sigma_dynamic_dphi = []
sigma_dynamic_dphi_err = []
pt_point = 0.5

for ifile in range(N_files):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    for icentr in range(N_centr):
        dynamic_dphi_icentr = array( 'd' )
        dynamic_dphi_err_icentr = array( 'd' )
        for ilayer in range(N_layers):
            dynamic_dphi_icentr.append(fit_functions[ifile][icentr][ilayer].Eval(pt_point)/(fit_functions[ifile][icentr][2].Eval(pt_point)+fit_functions[ifile][icentr][3].Eval(pt_point))*2)
            dynamic_dphi_err_icentr.append((fit_functions[ifile][icentr][ilayer].GetParError(0)*2)/(fit_functions[ifile][icentr][2].Eval(pt_point)+fit_functions[ifile][icentr][3].Eval(pt_point))*2)
        dynamic_dphi_ifile.append(dynamic_dphi_icentr)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_icentr)
    sigma_dynamic_dphi.append(dynamic_dphi_ifile)
    sigma_dynamic_dphi_err.append(dynamic_dphi_err_ifile)

print(sigma_dynamic_dphi)
sigma_dynamic_dphi[0][5][0]=0.88

[[array('d', [1.04143060716222, 1.0392026918946642, 1.0052119745306158, 0.994788025469384, 1.0015638460144076, 1.0010608391971239]), array('d', [1.0102115322546377, 0.9990764512417004, 0.9947396013477097, 1.0052603986522903, 1.0154026230291737, 1.01956918158816]), array('d', [1.0688503041783572, 1.031456565101357, 1.0116490117159354, 0.9883509882840644, 0.9435745724633866, 0.9083648696838792]), array('d', [0.9279143511761968, 0.9453019366500435, 0.991850049120956, 1.0081499508790441, 1.048860963298018, 1.0787241225177673]), array('d', [1.3372216214443686, 1.215542611700233, 1.0246532103665698, 0.9753467896334302, 0.9291645822945512, 0.9117579978560415]), array('d', [0.9730681628574145, 0.9548621111577752, 0.9804627144925224, 1.0195372855074774, 1.1593538676553907, 1.4008253636922408])]]


In [42]:
for ifile in range(N_files):
    h3=[]
    c4 = root.TCanvas(f"c4",f"c4",600,450)
    g_sigma_phi = [[[[]]*N_layers]*N_centr]*N_files
    min, max = -1, 5
    if iOption1==1: max, min = -1, 1
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(1):
        h3.append(Format_Pad_old( -0.03, 0.05, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h3[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for icentr in range(0,N_centr):

            g_sigma_phi[ifile][icentr][ilayer].append(root.TGraphErrors(N_layers, dphi, sigma_dynamic_dphi[ifile][icentr],dphi_err,sigma_dynamic_dphi_err[ifile][icentr]))
            
            Format_Graph(g_sigma_phi[ifile][icentr][ilayer][-1],20+int(icentr/4),1, colors[icentr], 3, colors[icentr],1,1)
            g_sigma_phi[ifile][icentr][ilayer][-1].Fit("pol2","","",)
            g_sigma_phi[ifile][icentr][ilayer][-1].Draw('sameP')
            legends[-1].AddEntry(g_sigma_phi[ifile][icentr][ilayer][-1],centralities[icentr],"p")

        legends[-1].Draw()
    c4.Draw()
    c4.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{iOption1}.png")

 FCN=0.290332 FROM MIGRAD    STATUS=CONVERGED      83 CALLS          84 TOTAL
                     EDM=5.14454e-11    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.00535e+00   1.97443e-02   7.80466e-06   5.86614e-04
   2  p1          -2.69320e+00   2.50750e+00   1.38448e-03  -1.81907e-06
   3  p2           2.41284e+02   4.36678e+02   1.71976e-01   1.11192e-08
 FCN=0.0892406 FROM MIGRAD    STATUS=CONVERGED      52 CALLS          53 TOTAL
                     EDM=1.00355e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.00090e+00   1.93429e-02   6.82323e-06   2.95262e-02
   2  p1           8.37997e-01   2.28455e+00   1.15450e-03   4.89889e-05
   3  p2           1.84329e+02

In [44]:
print("const float "+iOptames[iOption0]+f"_sigma_pt_params[{N_files}][{N_centr}][3]="+"{")
for fit_functions_ifile in fit_functions:
    print("{")
    for fits in fit_functions_ifile:
        print("{"+f"{round((fits[2].GetParameter(0)+fits[3].GetParameter(0))*0.5,6)}, {round((fits[2].GetParameter(1)+fits[3].GetParameter(1))*0.5,6)}, {round((fits[2].GetParameter(2)+fits[3].GetParameter(2))*0.5,6)}"+"},")
    print("},")      
print("};")
print("const float "+iOptames[iOption0]+f"_sigma_"+iOptames[iOption0]+f"_params[{N_files}][{N_centr}][3]="+"{")
for ifile in range(N_files):
    print("{")
    for icentr in range(N_centr):
        print("{"+f'{round(g_sigma_phi[ifile][icentr][0][icentr].GetFunction("pol2").GetParameter(0),6)},{round(g_sigma_phi[ifile][icentr][0][icentr].GetFunction("pol2").GetParameter(1),6)},{round(g_sigma_phi[ifile][icentr][0][icentr].GetFunction("pol2").GetParameter(2),6)}'+"},")
    print("},")
print("};")
print("const float "+iOptames[iOption0]+f"_mean_"+iOptames[iOption0]+f"_params[{N_files}][{N_centr}][2]="+"{")
for ifile in range(N_files):
    print("{")
    for icentr in range(N_centr):
        print("{"+f'{round(g_mean_phi[ifile][icentr][0][icentr].GetFunction("pol1").GetParameter(0),6)},{round(g_mean_phi[ifile][icentr][0][icentr].GetFunction("pol1").GetParameter(1),6)}'+"},")
    print("},")
print("};")



const float phi_sigma_pt_params[1][6][3]={
{
{0.001723, 0.007234, -4.9},
{0.001842, 0.007685, -4.95},
{0.001672, 0.010634, -2.45},
{0.001681, 0.009785, -2.45},
{0.001251, 0.004086, -2.5},
{0.001234, 0.003876, -2.5},
},
};
const float phi_sigma_phi_params[1][6][3]={
{
{1.005352,-2.693196,241.284224},
{1.000897,0.837997,184.329363},
{0.997244,-8.789062,-152.378999},
{0.998654,9.176908,27.358606},
{1.00838,-24.250899,1494.765133},
{1.006599,25.699848,1660.080676},
},
};
const float phi_mean_phi_params[1][6][2]={
{
{-0.000787,1.032902},
{0.00072,1.024858},
{-0.001464,1.070383},
{0.001105,1.050894},
{0.003485,2.391709},
{-0.004248,2.584326},
},
};
